# Bind Tools

In [1]:
pip install -q langchain langchain-openai langchain-community python-dotenv litellm

Note: you may need to restart the kernel to use updated packages.


## Token Setting

Get tokens from the pages:
- `OPENAI_API_KEY`: [OpenAI](https://platform.openai.com/settings/organization/api-keys)
- `HF_TOKEN`: [Huggingface](https://huggingface.co/settings/tokens)

In [2]:
import os
from dotenv import load_dotenv


env_file = ".env"
if os.path.exists(env_file):
    load_dotenv()
else:
    os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
    os.environ["HF_TOKEN"] = "<YOUR-HUGGINGFACE-API-KEY>"

## Model setting


In [3]:
from langchain_community.chat_models import ChatLiteLLM
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


# openai_llm = ChatLiteLLM(model="openai/gpt-3.5-turbo")
openai_llm = ChatOpenAI(model="gpt-3.5-turbo")
messages = [HumanMessage(content="what model are you")]
openai_llm.invoke(messages).pretty_print()

================================== Ai Message ==================================

I am a virtual assistant, powered by artificial intelligence.


In [4]:
hug_llm = ChatLiteLLM(
    model="huggingface/microsoft/Phi-3.5-mini-instruct",
    temperature=0.2,
)
messages = [HumanMessage(content="what model are you")]
hug_llm.invoke(messages).pretty_print()

================================== Ai Message ==================================

I am Phi, created by Microsoft. I'm not powered by a traditional model with hardware components but rather I'm a software-based AI language model. My architecture is based on Microsoft's research and is designed to understand and generate human-like text based on the prompts I receive.


## Functions

In this tutorial we will see how to select functions with OpenAI function call and call it on local.
We will use the [langchain tutorial](https://python.langchain.com/docs/how_to/tool_calling/) code for this tutorial.


Define functions with type hint and description.


In [5]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

Bind the defined functions with `bind_tools` attribute on llm.
When we ask query with this bind llm, llm will respond which function to call.


In [6]:
tools = [add, multiply]
llm_with_tools = openai_llm.bind_tools(tools)

response = llm_with_tools.invoke("What is 3 * 12?")
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  multiply (call_3YovBgEqlfpYIi8pm2HHRZuy)
 Call ID: call_3YovBgEqlfpYIi8pm2HHRZuy
  Args:
    a: 3
    b: 12


Returned response's has `tool_calls` attribute.
This tools mean that llm ask to client to run those tools.


In [7]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_3YovBgEqlfpYIi8pm2HHRZuy',
  'type': 'tool_call'}]

This tool calls can be multiple.
This is example query to get multiple tool calls.


In [8]:
response = llm_with_tools.invoke("What is 3 * 12? Also, what is 11 + 49?")
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  multiply (call_FwR3ip6Bim8wbFvwCoXKmzng)
 Call ID: call_FwR3ip6Bim8wbFvwCoXKmzng
  Args:
    a: 3
    b: 12
  add (call_Wj4FWC26qkJAXgrzalF7NxQj)
 Call ID: call_Wj4FWC26qkJAXgrzalF7NxQj
  Args:
    a: 11
    b: 49


In [9]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_FwR3ip6Bim8wbFvwCoXKmzng',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_Wj4FWC26qkJAXgrzalF7NxQj',
  'type': 'tool_call'}]

If there is not valid tools to call it will respond that there is not valid one.
And `tool_calls` will be empty list.


In [10]:
query = "What is?"

response = llm_with_tools.invoke(query)
response.pretty_print()

================================== Ai Message ==================================

It seems like you haven't provided any specific question or task. How can I assist you today?


In [11]:
response.tool_calls

[]

Now let's run the selected tool.


## OpenAI

What happen when we use `bind_tools`?
Before tutorial, We already check how the tool is calling with structured output.
This function call is basically same, but only we have convert tools as json format like this:

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "unit": {"type": "string", "enum": ["c", "f"]},
                },
                "required": ["location", "unit"],
                "additionalProperties": False,
            },
        },
    }
]
```

And langchain helps us to make this json type easily.


In [12]:
from openai import OpenAI

client = OpenAI()

In [13]:
from langchain_core.utils.function_calling import convert_to_openai_tool


openai_tools = [convert_to_openai_tool(tool) for tool in tools]
openai_tools

[{'type': 'function',
  'function': {'name': 'add',
   'description': 'Add two integers.',
   'parameters': {'properties': {'a': {'description': 'First integer',
      'type': 'integer'},
     'b': {'description': 'Second integer', 'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'multiply',
   'description': 'Multiply two integers.',
   'parameters': {'properties': {'a': {'description': 'First integer',
      'type': 'integer'},
     'b': {'description': 'Second integer', 'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}}]

In [14]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "What is 3 * 12?",
        },
    ],
    tools=openai_tools,
)
event = completion.choices[0].message
print(event)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_P6Kf9IYrgEo6qN3eszpAy6HQ', function=Function(arguments='{"a":3,"b":12}', name='multiply'), type='function')])


In [15]:
event.tool_calls

[ChatCompletionMessageToolCall(id='call_P6Kf9IYrgEo6qN3eszpAy6HQ', function=Function(arguments='{"a":3,"b":12}', name='multiply'), type='function')]

In [16]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "What is?",
        },
    ],
    tools=openai_tools,
)
event = completion.choices[0].message
print(event)

ChatCompletionMessage(content='It seems that your question is incomplete. Could you please provide more context or clarify what you would like to know?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


## Huggingface

Now we will make this process manual for the unsupported llm endpoint.

To make this we will use `langchain.tools`.
Just decorating the function we can get format that can use in prompt.


In [17]:
# tool
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

This converted tool has attributes like below:


In [18]:
langchain_tools = [add, multiply]

for tool in langchain_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)

add
Add two integers.

    Args:
        a: First integer
        b: Second integer
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
multiply
Multiply two integers.

    Args:
        a: First integer
        b: Second integer
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


Render tools to use in prompt.


In [19]:
from langchain_core.tools import render_text_description


rendered_tools = render_text_description(langchain_tools)
tool_names = [tool.name for tool in langchain_tools]
print(rendered_tools)

add(a: int, b: int) -> int - Add two integers.

    Args:
        a: First integer
        b: Second integer
multiply(a: int, b: int) -> int - Multiply two integers.

    Args:
        a: First integer
        b: Second integer


This is prompt that we will use.

Prompt generates a json format with key, name and args.
`name` key is name of the tool we will use, but if there is not proper tool to call it will generate refusal.
We will use this refusal later, to handle edge case.
And `args` are dict type that contains values which is used to invoke tool.


In [20]:
from langchain_core.prompts import ChatPromptTemplate

# https://smith.langchain.com/hub/hwchase17/react-chat-json
system_prompt = """\
TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question.
The tools the human can use are:

{rendered_tools}

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of three formats:

**Option 1:**
Use this if you cannot determine which tool to use.
Markdown code snippet formatted in the following schema:

{{
    "name": "refusal"
}}


**Option 2:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

{{
    "name": string, \\ The action to take. Must be one of {tool_names}
    "args": dict, \\ The input to the action described {rendered_tools}
}}

USER'S INPUT
--------------------

Here is the user's input (remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else):
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)
prompt = prompt.partial(
    rendered_tools=rendered_tools,
    tool_names=tool_names,
)

This will return with json type.
So we can easily parse this with `JsonOutputParser`.


In [21]:
chain = prompt | hug_llm
response = chain.invoke({"What is 3 * 12?"})
response.pretty_print()

================================== Ai Message ==================================

{
    "name": "multiply",
    "args": {
        "a": 3,
        "b": 12
    }
}


In [22]:
from langchain_core.output_parsers import JsonOutputParser


chain = prompt | hug_llm | JsonOutputParser()
response = chain.invoke({"What is 3 * 12?"})
response

{'name': 'multiply', 'args': {'a': 3, 'b': 12}}

With [langchain tutorial](https://python.langchain.com/docs/how_to/tools_prompting/), we will define a function `invoke_tool` to call the proper tool and give its output to chain result.


In [23]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    if tool_call_request["name"] == "refusal":
        return None
    tool_name_to_tool = {tool.name: tool for tool in langchain_tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["args"], config=config)

This function can use like below:


In [24]:
invoke_tool({"name": "multiply", "args": {"a": 3, "b": 12}})

36

Now, let's make a chain to use `invoke_tool` function.


In [25]:
chain = prompt | hug_llm | JsonOutputParser() | invoke_tool
chain.invoke({"input": "What is 3 * 12?"})

36

To make this as a chain we can use `RunnablePassthrough` function.


In [26]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt
    | hug_llm
    | JsonOutputParser()
    | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "What is 3 * 12?"})

{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'output': 36}

We didn't make our prompt to return multiple tool calling, so we cannot use multiple tools here.

And this is why functions like `JsonOutputToolsParser` has an argument like `first_tool_only`.


In [27]:
chain.invoke({"input": "What is 3 * 12? Also, what is 11 + 49?"})

{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'output': 36}

And you can see that our prompt is using only first tool.


In [28]:
chain.invoke({"input": "What is 11 + 49? Also, what is 3 * 12?"})

{'name': 'add', 'args': {'a': 11, 'b': 49}, 'output': 60}

This is a case with error handling like not proper tool to call:


In [29]:
chain.invoke({"input": "What is?"})

{'name': 'refusal', 'output': None}

## Multiple Tools

Now we will fix prompt little bit to support multiple tool calling.
Return values with list and add Option 3 for multiple tools.


In [30]:
from langchain_core.prompts import ChatPromptTemplate

# https://smith.langchain.com/hub/hwchase17/react-chat-json
system_prompt = """
TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question.
The tools the human can use are:

{rendered_tools}

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of three formats:

**Option 1:**
Use this if you cannot determine which tool to use.
Markdown code snippet formatted in the following schema:

[
    {{
        "name": "refusal"
    }}
]


**Option 2:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

[
    {{
        "name": string, \\ The action to take. Must be one of {tool_names}
        "args": dict, \\ The input to the action described {rendered_tools}
    }}
]

**Option 3:**
Use this if you want the human to use multiple tools.
Markdown code snippet formatted in the following schema:

[
    {{
        "name": string, \\ The action to take. Must be one of {tool_names}
        "args": dict, \\ The input to the action described {rendered_tools}
    }},
    {{
        "name": string, \\ The action to take. Must be one of {tool_names}
        "args": dict, \\ The input to the action described {rendered_tools}
    }}
]

USER'S INPUT
--------------------

Here is the user's input (remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else):
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)
prompt = prompt.partial(
    rendered_tools=rendered_tools,
    tool_names=tool_names,
)

Now it will return like this:


In [31]:
from langchain_core.output_parsers import JsonOutputParser


chain = prompt | hug_llm | JsonOutputParser()
response = chain.invoke({"What is 3 * 12?"})
response

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}}]

To use this we have to fix the schema.

First write `ToolCall` which is same as before example.
And write `ToolCallRequests` which contains list of `ToolCall`.

In [32]:
from typing import Any, Dict, Optional

from langchain_core.runnables import RunnableConfig


def invoke_tools(
    tool_call_requests: Dict[Any, Any], config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_requests: a list with dicts and dict contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    results = []
    for tool_call_request in tool_call_requests["tool_calls"]:
        if tool_call_request["name"] == "refusal":
            return None

        tool_name_to_tool = {tool.name: tool for tool in langchain_tools}
        name = tool_call_request["name"]
        requested_tool = tool_name_to_tool[name]
        response = requested_tool.invoke(tool_call_request["args"], config=config)
        results += [response]
    return results

`RunnablePassthrough` function needs input as a dictionary.
So we will add a dictionary to the parser.
You can see that output has been changed as list.


In [33]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt
    | hug_llm
    | {"tool_calls": JsonOutputParser()}
    | RunnablePassthrough.assign(output=invoke_tools)
)
chain.invoke({"input": "What is 3 * 12?"})

{'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 12}}],
 'output': [36]}

Now we can handle multiple tool calling.


In [34]:
chain.invoke({"input": "What is 3 * 12? Also, what is 11 + 49?"})

{'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 12}},
  {'name': 'add', 'args': {'a': 11, 'b': 49}}],
 'output': [36, 60]}

And also order is preserved.


In [35]:
chain.invoke({"input": "What is 11 + 49? Also, what is 3 * 12?"})

{'tool_calls': [{'name': 'add', 'args': {'a': 11, 'b': 49}},
  {'name': 'multiply', 'args': {'a': 3, 'b': 12}}],
 'output': [60, 36]}

This is a case with error handling like not proper tool to call:


In [36]:
chain.invoke({"input": "What is?"})

{'tool_calls': [{'name': 'refusal'}], 'output': None}

## Thoughts

Now lets use the tool output to generate answer with user query.
To generate answer we have to keep all messages on the chain.


In [37]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage


query = "What is 3 * 12? Also, what is 11 + 49?"

chain = prompt | hug_llm | {"tool_calls": JsonOutputParser()}

messages = [HumanMessage(query)]
ai_msg = chain.invoke(messages)
print(ai_msg["tool_calls"])
messages.append(AIMessage(str(ai_msg)))

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}}, {'name': 'add', 'args': {'a': 11, 'b': 49}}]


Stack tool results to the message:

In [38]:
for tool_call in ai_msg["tool_calls"]:
    name = tool_call["name"].lower()
    selected_tool = {"add": add, "multiply": multiply}[name]
    tool_msg = selected_tool.invoke(tool_call["args"])
    messages.append(
        ToolMessage(content=tool_msg, name=name, tool_call_id=f"tmp-{name}-{tool_msg}")
    )

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="{'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 12}}, {'name': 'add', 'args': {'a': 11, 'b': 49}}]}", additional_kwargs={}, response_metadata={}),
 ToolMessage(content='36', name='multiply', tool_call_id='tmp-multiply-36'),
 ToolMessage(content='60', name='add', tool_call_id='tmp-add-60')]

And we can ask llm to answer with this results.

In [39]:
hug_llm.invoke(messages).pretty_print()

================================== Ai Message ==================================

3 * 12 equals 36.

11 + 49 equals 60.

So, the results are:
- 3 * 12 = 36
- 11 + 49 = 60

In terms of the tool calls:
- The 'multiply' tool call with arguments 3 and 12 gives the result 36.
- The 'add' tool call with arguments
